In [1]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv('интерполяция.csv', engine='python', sep=';', encoding='utf-8')
del df['Unnamed: 0.1']


In [11]:
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 500) 
pd.set_option('display.width', 1000)

In [13]:
features_to_check = list(df)[19:-3]
features_to_check

['Чистые активы, RUB',
 'Счета в Банке России, RUB',
 'Коррсчета (НОСТРО), RUB',
 'Ценные бумаги, RUB',
 'Кредиты, RUB',
 'Кредиты организациям, RUB',
 'Кредиты физическим лицам, RUB',
 'Кредиты другим банкам (МБК), RUB',
 'Основные средства, RUB',
 'Учтённые векселя, RUB',
 'Прибыль (убыток) до налогообложения, RUB',
 'Коррсчета (ЛОРО), RUB',
 'Кредиты других банков (МБК), RUB',
 'Средства клиентов, RUB',
 'Средства организаций на расчетных счетах, RUB',
 'Депозиты юридических лиц, RUB',
 'Вклады физических лиц, RUB',
 'Облигации, RUB',
 'Векселя, RUB',
 'Резервы на возможные потери, RUB',
 'Капитал, RUB',
 'Коэффициент покрытия Кредитного портфеля, %',
 'Коэффициент обеспечения Кредитного портфеля, %',
 'Коэффициент просроченной задолженности в Кредитном портфеле, %']

In [17]:
def get_nans_intervals(v):
    # это слишком гениально, чтобы объяснять
    # возвращает все интервалы из идущих подряд нанов в векторе
    m = np.concatenate(([True], ~np.isnan(v), [True]))
    return np.flatnonzero(m[1:] != m[:-1]).reshape(-1, 2)

In [18]:
l = pd.Series([np.nan, np.nan, 2, np.nan, 4, np.nan, np.nan])


In [20]:
l

0    NaN
1    NaN
2    2.0
3    NaN
4    4.0
5    NaN
6    NaN
dtype: float64

In [24]:
intervals1 = get_nans_intervals(l)
print(intervals1[1])
print(intervals1[0])
print(intervals1[1] - intervals1[0])


[3 4]
[0 2]
[3 2]


In [25]:
intervals1

array([[0, 2],
       [3, 4],
       [5, 7]], dtype=int64)

In [21]:
def get_len(interval):
    return interval[1] - interval[0]

In [22]:
intervals2 = get_len(intervals1)
intervals2

array([3, 2], dtype=int64)

In [50]:
# чистим те, что без данных за весь период

features_to_check = list(df)[19:-3]

indexes_to_drop = set()
for col_name in features_to_check:
    for i in df['index'].unique():
        # если банк уже в списке на удаление, проверять его не имеет смысла
        if i not in indexes_to_drop:
            column = df[df['index'] == i][col_name]
            nan_intervals = get_nans_intervals(column)
            # банк нужно удалить, если есть всего один интервал из nan'ов и он по длине равен длине самой колонки
            if len(nan_intervals == 1) and get_len(nan_intervals[0]) == len(column):
                indexes_to_drop.add(i)

In [51]:
len(indexes_to_drop)

110

In [52]:
df = df[~df['index'].isin(indexes_to_drop)]

In [56]:
row_indexes_to_drop = []
for i in df['index'].unique():
    for col_name in features_to_check:
        subdf = df[df['index'] == i]
        column = subdf[col_name]
        nan_intervals = get_nans_intervals(column)
        for nan_interval in nan_intervals:
            if get_len(nan_interval) <= 4:
                # если в nan_interval содержится 0, значит, есть кусок пропусков в начале
                if 0 in nan_interval:
                    df[df['index'] == i][col_name].bfill()
                # аналогично с len(column) и концом
                elif len(column) in nan_interval:
                    df[df['index'] == i][col_name].ffill()
            # если длина пропуска больше 4 и банк действующий, добавляем индексы nan'ов в список на удаление
            elif subdf.iloc[0].status:
                    row_indexes_to_drop.extend(subdf.index[nan_interval[0]:nan_interval[1]])
                
                
# Печатает индексы и колонки, в которых есть пропуски посередине        
#         try:
#             assert len(nan_intervals) < 3
#             if len(nan_intervals) == 1:
#                 assert 0 in nan_intervals[0] or len(df[df['index'] == i][col]) in nan_intervals[0]
#             elif len(nan_intervals) == 2:
#                 assert 0 in nan_intervals[0] and len(df[df['index'] == i][col]) in nan_intervals[1]
#         except AssertionError:
#             print(i, col_name, len(column), nan_intervals)

In [58]:
len(row_indexes_to_drop)

9053

In [65]:
df = df[~df.index.isin(row_indexes_to_drop)]
df.shape

(45861, 45)

In [66]:
df.to_csv('final.csv', sep=';', index=None)

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('final.csv', engine='python', sep=';', encoding='utf-8')

In [12]:
df.head(500)

,Unnamed: 0,index,year,month,Наименование,Регистрационный номер,Наименование полное,Адрес (место нахождения),Руководитель ФИО,Дата регистрации,Возраст банка,Дата ликвидации,Статус,Банковский идентификационный номер,Регион регистрации,Организационно-правовая форма,Форма собственности,"Уставный капитал, RUB",Важная информация,"Чистые активы, RUB","Счета в Банке России, RUB","Коррсчета (НОСТРО), RUB","Ценные бумаги, RUB","Кредиты, RUB","Кредиты организациям, RUB","Кредиты физическим лицам, RUB","Кредиты другим банкам (МБК), RUB","Основные средства, RUB","Учтённые векселя, RUB","Прибыль (убыток) до налогообложения, RUB","Коррсчета (ЛОРО), RUB","Кредиты других банков (МБК), RUB","Средства клиентов, RUB","Средства организаций на расчетных счетах, RUB","Депозиты юридических лиц, RUB","Вклады физических лиц, RUB","Облигации, RUB","Векселя, RUB","Резервы на возможные потери, RUB","Капитал, RUB","Коэффициент покрытия Кредитного портфеля, %","Коэффициент обеспечения Кредитного портфеля, %","Коэффициент просроченной задолженности в Кредитном портфеле, %",row_index,year_month,status
0,0,1,2013,1,"ИВАНОВО, АО КБ",1763.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК ""ИВАНОВО""","153002, Ивановская обл, г. Иваново, проспект Л...",Морковкин Артемий Львович,13.04.1992,"27,0",NaN,В состоянии банкротства,42406705.0,Ивановская область,Непубличные акционерные общества,Совместная частная и иностранная собственность,142 500 000,Отозвана Базовая лицензия на осуществление бан...,2.661870e+09,4.550760e+08,5.593600e+07,8.012100e+07,1.752808e+09,1.182441e+09,5.319790e+08,5.500000e+05,1.217200e+07,0.000000e+00,2.027600e+07,0.000000e+00,6.333400e+07,2.115523e+09,4.871300e+08,1.156880e+08,1.509481e+09,0.0,3.162000e+06,2.019210e+08,3.453070e+08,8.840000,128.170000,2.510000,0,2013_01,0
1,1,1,2013,2,"ИВАНОВО, АО КБ",1763.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК ""ИВАНОВО""","153002, Ивановская обл, г. Иваново, проспект Л...",Морковкин Артемий Львович,13.04.1992,"27,0",NaN,В состоянии банкротства,42406705.0,Ивановская область,Непубличные акционерные общества,Совместная частная и иностранная собственность,142 500 000,Отозвана Базовая лицензия на осуществление бан...,2.661870e+09,4.550760e+08,5.593600e+07,8.012100e+07,1.752808e+09,1.182441e+09,5.319790e+08,5.500000e+05,1.217200e+07,0.000000e+00,2.027600e+07,0.000000e+00,6.333400e+07,2.115523e+09,4.871300e+08,1.156880e+08,1.509481e+09,0.0,3.162000e+06,2.019210e+08,3.453070e+08,8.840000,128.170000,2.510000,1,2013_02,0
2,2,1,2013,3,"ИВАНОВО, АО КБ",1763.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК ""ИВАНОВО""","153002, Ивановская обл, г. Иваново, проспект Л...",Морковкин Артемий Львович,13.04.1992,"27,0",NaN,В состоянии банкротства,42406705.0,Ивановская область,Непубличные акционерные общества,Совместная частная и иностранная собственность,142 500 000,Отозвана Базовая лицензия на осуществление бан...,2.661870e+09,4.550760e+08,5.593600e+07,8.012100e+07,1.752808e+09,1.182441e+09,5.319790e+08,5.500000e+05,1.217200e+07,0.000000e+00,2.027600e+07,0.000000e+00,6.333400e+07,2.115523e+09,4.871300e+08,1.156880e+08,1.509481e+09,0.0,3.162000e+06,2.019210e+08,3.453070e+08,8.840000,128.170000,2.510000,2,2013_03,0
3,3,1,2013,4,"ИВАНОВО, АО КБ",1763.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК ""ИВАНОВО""","153002, Ивановская обл, г. Иваново, проспект Л...",Морковкин Артемий Львович,13.04.1992,"27,0",NaN,В состоянии банкротства,42406705.0,Ивановская область,Непубличные акционерные общества,Совместная частная и иностранная собственность,142 500 000,Отозвана Базовая лицензия на осуществление бан...,2.141722e+09,9.351600e+07,6.826900e+07,6.446200e+07,1.660469e+09,1.192624e+09,3.912310e+08,5.052500e+07,1.200500e+07,5.069000e+06,5.939000e+06,0.000000e+00,2.483400e+07,1.668166e+09,2.432380e+08,1.136440e+08,1.302956e+09,0.0,1.406500e+07,1.758850e+08,3.345390e+08,8.100000,140.510000,2.700000,3,2013_04,0
4,4,1,2013,5,"ИВАНОВО, АО КБ",1763.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК ""ИВАНОВО""","153002, Ивановская обл, г. Иваново, проспект Л...",Морковк

In [6]:
df4 = df.groupby(['index', 'year', 'month']).apply(lambda x: x.isna().sum()*100/len(x))

In [7]:
df4.to_csv('4_final.csv', sep=';', encoding='cp1251')

In [30]:
df.loc[4430:4900]

,Unnamed: 0,index,year,month,Наименование,Регистрационный номер,Наименование полное,Адрес (место нахождения),Руководитель ФИО,Дата регистрации,Возраст банка,Дата ликвидации,Статус,Банковский идентификационный номер,Регион регистрации,Организационно-правовая форма,Форма собственности,"Уставный капитал, RUB",Важная информация,"Чистые активы, RUB","Счета в Банке России, RUB","Коррсчета (НОСТРО), RUB","Ценные бумаги, RUB","Кредиты, RUB","Кредиты организациям, RUB","Кредиты физическим лицам, RUB","Кредиты другим банкам (МБК), RUB","Основные средства, RUB","Учтённые векселя, RUB","Прибыль (убыток) до налогообложения, RUB","Коррсчета (ЛОРО), RUB","Кредиты других банков (МБК), RUB","Средства клиентов, RUB","Средства организаций на расчетных счетах, RUB","Депозиты юридических лиц, RUB","Вклады физических лиц, RUB","Облигации, RUB","Векселя, RUB","Резервы на возможные потери, RUB","Капитал, RUB","Коэффициент покрытия Кредитного портфеля, %","Коэффициент обеспечения Кредитного портфеля, %","Коэффициент просроченной задолженности в Кредитном портфеле, %",row_index,year_month,status
4430,4430,86,2013,2,"ВОЛОГДАБАНК, АО",992.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛОГДАБАНК""","160001, Вологодская обл., г. Вологда, ул. Мира...",Морсков Дмитрий Геннадьевич,28.11.1990,"28,0",NaN,Ликвидируется,41909717.0,Вологодская область,Непубличные акционерные общества,Частная собственность,60 119 723,"Находится в стадии ликвидации от 31.01.2018, О...",1.279828e+09,4.277200e+07,1.405190e+08,0.000000e+00,8.512490e+08,6.913690e+08,1.598800e+08,0.000000e+00,1.799050e+08,0.0,7.592490e+08,2.440000e+05,1.779749e+09,3.387882e+10,8.359068e+09,8.094636e+09,1.719016e+10,1.500000e+09,3.082822e+09,2.175130e+09,5.673338e+09,5.740000,115.500000,2.400000,6121,2013_02,0
4431,4431,86,2013,3,"ВОЛОГДАБАНК, АО",992.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛОГДАБАНК""","160001, Вологодская обл., г. Вологда, ул. Мира...",Морсков Дмитрий Геннадьевич,28.11.1990,"28,0",NaN,Ликвидируется,41909717.0,Вологодская область,Непубличные акционерные общества,Частная собственность,60 119 723,"Находится в стадии ликвидации от 31.01.2018, О...",1.279828e+09,4.277200e+07,1.405190e+08,0.000000e+00,8.512490e+08,6.913690e+08,1.598800e+08,0.000000e+00,1.799050e+08,0.0,7.592490e+08,2.440000e+05,1.779749e+09,3.387882e+10,8.359068e+09,8.094636e+09,1.719016e+10,1.500000e+09,3.082822e+09,2.175130e+09,5.673338e+09,5.740000,115.500000,2.400000,6122,2013_03,0
4432,4432,86,2013,4,"ВОЛОГДАБАНК, АО",992.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛОГДАБАНК""","160001, Вологодская обл., г. Вологда, ул. Мира...",Морсков Дмитрий Геннадьевич,28.11.1990,"28,0",NaN,Ликвидируется,41909717.0,Вологодская область,Непубличные акционерные общества,Частная собственность,60 119 723,"Находится в стадии ликвидации от 31.01.2018, О...",1.379860e+09,6.880000e+07,2.006900e+08,0.000000e+00,8.372360e+08,6.801590e+08,1.570770e+08,0.000000e+00,1.982740e+08,0.0,7.078700e+07,1.750000e+05,9.355790e+08,2.828520e+10,6.727886e+09,6.633370e+09,1.449759e+10,0.000000e+00,4.283290e+09,2.029947e+09,4.513803e+09,6.180000,126.690000,2.960000,6123,2013_04,0
4433,4433,86,2013,5,"ВОЛОГДАБАНК, АО",992.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛОГДАБАНК""","160001, Вологодская обл., г. Вологда, ул. Мира...",Морсков Дмитрий Геннадьевич,28.11.1990,"28,0",NaN,Ликвидируется,41909717.0,Вологодская область,Непубличные акционерные общества,Частная собственность,60 119 723,"Находится в стадии ликвидации от 31.01.2018, О...",1.563959e+09,5.281000e+07,3.233370e+08,0.000000e+00,9.143050e+08,6.774890e+08,1.568160e+08,8.000000e+07,1.960650e+08,0.0,1.845710e+08,1.320000e+05,1.130619e+09,2.949265e+10,6.740264e+09,7.598038e+09,1.478532e+10,0.000000e+00,4.280872e+09,2.152012e+09,5.235200e+09,6.340000,125.780000,2.690000,6124,2013_05,0
4434,4434,86,2013,6,"ВОЛОГДАБАНК, АО",992.0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВОЛОГДАБАНК""","160001, Вологодская обл., г. Вологда, ул. Мира...",Морсков Дмитрий Геннадьевич,28.11.1990,"28,0",NaN,Ликвидируется,41909717.0,Вологодская область,Непубличные акционерные общества,Частная собственность,60 1

In [28]:
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 500) 
pd.set_option('display.width', 1000)